In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,roc_auc_score
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
import pandas as pd
item_categories = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
sales_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv", index_col='ID')

In [ ]:
##Cleaning object data into dat_time and parsing
sales_train['date']=pd.to_datetime(sales_train['date'],format='%d.%m.%Y')

sales_train=sales_train.assign(day=sales_train['date'].dt.day,
                  month=sales_train['date'].dt.month,
                  year=sales_train['date'].dt.year)

##Multiplying price_per_item by number_of_items
#sales_train['item_price']=sales_train['item_price']*sales_train['item_cnt_day']

In [ ]:
#attaching data from items to sales_train
a=items.set_index(['item_id'])
sales_train['item_name']=sales_train['item_id'].map(a.item_name)
sales_train['item_category_id']=sales_train['item_id'].map(a.item_category_id)

#attaching data from item_categories to sales_train
a=item_categories.set_index(['item_category_id'])
sales_train['item_category_name']=sales_train['item_category_id'].map(a.item_category_name)

# We are not attaching shop_names for now as I  dont feel  it would be useful

## DATA cleaning

In [ ]:
def data_cleaner(df):
    ##selecting non-string cols
    X=df[['date_block_num','shop_id','item_id','item_price']]
    y=df['item_cnt_day']

    ##grouping by per_month, per_shop, per_item
    all_data=X.join(y)
    a=all_data.groupby(['date_block_num','shop_id','item_id']).sum()
    a['item_price']=all_data.groupby(['date_block_num','shop_id','item_id'])['item_price'].mean()

    ##breaking GroupBy multiIndex and renaming the groupby-summed column
    a.reset_index(inplace=True)
    a.rename({'item_cnt_day':'item_cnt_month'},inplace=True,axis=1)
    all_data=a

    ##redefining X and y
    X=all_data.drop('item_cnt_month',axis=1)
    y=all_data['item_cnt_month']

    ##Mapping item_categories_id to all_data
    all_data['item_category_id']=all_data['item_id'].map(items['item_category_id'])

    ##rearranging all_data column sequence
    all_data=all_data[['date_block_num','shop_id','item_id','item_category_id','item_price','item_cnt_month']]
    
    return all_data


In [ ]:
def model_evaluator(ad):
    model=LinearRegression()
    curr_block_num=33
    X_train = ad.loc[ad['date_block_num']<33].drop(['item_cnt_month'],axis=1)
    y_train = ad.loc[ad['date_block_num']<33]['item_cnt_month']
    X_valid = ad.loc[ad['date_block_num']==33].drop(['item_cnt_month'],axis=1)
    y_valid = ad.loc[ad['date_block_num']==33]['item_cnt_month']
    model.fit(X_train,y_train)
    preds_valid=model.predict(X_valid)
    
    return np.sqrt(mean_squared_error(y_valid,preds_valid))

In [ ]:
all_data=data_cleaner(sales_train)

In [ ]:
model_evaluator(all_data)

In [ ]:
all_data.head()

# EDA

In [ ]:
## CHecking for NANs
all_data.isnull().any()

### It can be observed that item count peaks every november

In [ ]:
## time variation of monthly sales of all shops grouped
fig=plt.figure(figsize=(10,5))

a=all_data.groupby(['date_block_num'])['item_cnt_month'].sum()
plt.plot(a.index,a.values)
plt.xlabel('date_block_num')
plt.ylabel('item_cnt_month')

### EDA item_price distribution

In [ ]:
#boxplotting item_price
g=sns.FacetGrid(sales_train)
g.map(sns.boxplot,x=sales_train['item_price'])

In [ ]:
print('max item price',sales_train['item_price'].max())
print('min item price',sales_train['item_price'].min())

In [ ]:
max_id=sales_train['item_price'].idxmax()
min_id=sales_train['item_price'].idxmin()
a=sales_train.drop([max_id,min_id])
all_data=data_cleaner(a)

In [ ]:
#rechecking distribuion
print('min item price',a.item_price.min())
print('max item price',a.item_price.max())
sns.boxplot(a.item_price)

In [ ]:
##as the sample is small hence we can drop columns with item_price>40000
a.loc[a.item_price>40000].shape

In [ ]:
a1=a.drop(a.loc[a.item_price>40000].index,axis=0,inplace=False)
all_data=data_cleaner(a1)

### EDA observation- sudden spikes

In [ ]:
## we can notice the existence of outliers from this scatter plot
plt.figure(figsize=(10,4))
sns.scatterplot(all_data.index,all_data['item_cnt_month'].values)
plt.xlabel('index')
plt.ylabel('item_cnt_month')

In [ ]:
##EDA observation 1-sudden  spike  in itemcounts
plt.figure(figsize=(10,4))
plt.plot(all_data.index,all_data['item_cnt_month'].values)
plt.ylabel('item_cnt_month')
plt.xlabel('index')

In [ ]:
g=sns.FacetGrid(all_data,height=4, aspect=3)
g.map(sns.boxplot,x=all_data['item_cnt_month'])
## hence we can exclude data with monthly sales count more than 1000

In [ ]:
sns.set_style(style='whitegrid')
g=sns.FacetGrid(all_data,height=4, aspect=3)
g.map(sns.boxplot,x=all_data['shop_id'],y=all_data['item_cnt_month'])
## hence we can exclude data with monthly sales count more than 1000

In [ ]:
## dropping all rows with item_cnt_month>1000 (because they are very scarce)
all_data=all_data.loc[all_data['item_cnt_month']<=1000]

In [ ]:
##rechecking for outliers
g=sns.FacetGrid(all_data,height=4, aspect=3)
g.map(sns.boxplot,x=all_data['item_cnt_month'])

### EDA checking sales of different item_categories

In [ ]:
sns.distplot(all_data['item_category_id'])

## --------- end of EDA ------------

#### we have brought the MSE from 211 to 50.67 by removing the outliers

In [ ]:
model_evaluator(all_data)

## ENCODING

In [ ]:
## so far we have only used numeric data
all_data.head()

In [ ]:
a=all_data.groupby('item_category_id').sum()['item_cnt_month']
sns.pointplot(a.index,a.values)

In [ ]:
a=all_data.groupby('shop_id').sum()['item_cnt_month']
sns.pointplot(a.index,a.values)

In [ ]:
#pickling
filename='cleaned_numeric.pickle'
outfile=open(filename,'wb')
pickle.dump(all_data,outfile)

In [ ]:
filename='cleaned_numeric.pickle'
infile=open(filename,'rb')
all_data=pickle.load(infile)

In [ ]:
all_data.to_pickle('cleaned_numeric1.pkl')

In [ ]:
all_data=pd.read_pickle('cleaned_numeric1.pkl')

### part 2

In [ ]:
index_cols=['shop_id','item_id','date_block_num']

In [ ]:
all_data

In [ ]:
all_data1=all_data[['shop_id','item_id','date_block_num','item_cnt_month']]

### I checked for outliers and found one! But then deecided to look at the boxplot and the boxplot for the particular item, seems like its not a misentry and the item has actually sold a lot

In [ ]:
## outlier exist when grouped by and summed
a=all_data1.groupby('item_id').sum()
sns.boxplot(a.item_cnt_month)

In [ ]:
#descending sort to check outliers
all_data.sort_values(by='item_cnt_month',ascending=False)

In [ ]:
##boxplot to see if the item_id has some vague entry
sns.boxplot(all_data.loc[all_data['item_id']==20949]['item_cnt_month'])

In [ ]:
## checking presence of outliers when no grouping and summing
sns.boxplot(all_data['item_cnt_month'])

In [ ]:
## since no outliers are present, lets check whether mean or sum encoding is better
a=all_data1.groupby('item_id').mean()
plt.scatter(x=a.index,y=a.item_cnt_month.values)

In [ ]:
#it can be seen that sum_encoding is not very helpful in distiguishing categories, hence we will do mean encoding
a=all_data1.groupby('item_id').sum()
plt.scatter(x=a.index,y=a.item_cnt_month.values)

We will do unregularized mean encoding

In [ ]:
drop_index=all_data1.loc[all_data['date_block_num']==33].index
a=all_data1.drop(drop_index,axis=0)

In [ ]:
a1=a.groupby('item_id').mean()['item_cnt_month']

In [ ]:
all_data1['item_id_mean']=all_data1['item_id'].map(a1)

In [ ]:
all_data['item_id_mean']=all_data1['item_id'].map(a1)

In [ ]:
all_data

In [ ]:
all_data1.item_id_mean.isnull().sum()